In [11]:
%reload_ext autoreload
%autoreload 3
import os

import xarray as xr
from pathlib import Path
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder

from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from dvclive.lightning import DVCLiveLogger

from src.data.julia2018 import Julia2018DataModule
from src.acnets import ACNets


# force running in the project directory (e.g., via nbconvert)
if 'notebooks' in os.getcwd():
    %cd ..

In [12]:
datamodule = Julia2018DataModule(segment_size=31, split_on='sequence', batch_size=96, shuffle=False)
datamodule.prepare_data()
n_subjects = len(datamodule.subjects)
n_inputs = datamodule.n_features

In [13]:

n_features = 32
hidden_size = n_features

model = ACNets(n_inputs, n_features, hidden_size, n_subjects)
# model = torch.compile(model)  # is not supported on GTX 1070

trainer = pl.Trainer(
    max_epochs=1000, accelerator='auto', log_every_n_steps=1,
    logger=[DVCLiveLogger(save_dvc_exp=True, report='md'),
            TensorBoardLogger('.')]
    )
trainer.fit(model, datamodule=datamodule)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/morteza/micromamba/envs/acnets-multihead/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/morteza/workspace/ACNets-MultiHead/DvcLiveLogger/dvclive_run/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | feature_extractor | FeatureExtractor | 76.1 K
1 | encoder           | SeqAutoEncoder   | 18.0 K
2 | head_cls          | Sequential       | 562   
3 | head_subj_cls     | Linear           | 1.1 K 
-------------------------------------------------------
95.7 K    Trainable params
0         Non-trainable params
95.7 K    Total params
0.383     Total es

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


Updating lock file 'dvc.lock'


INFO:dvc.dvcfile:Updating lock file 'dvc.lock'
INFO:dvclive:To run with DVC, add this to /home/morteza/workspace/ACNets-MultiHead/dvc.yaml:
stages:
  dvclive:
    cmd: <python my_code_file.py my_args>
    deps:
    - <my_code_file.py>

